In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
#import fasttext
from tqdm import tqdm
#from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
#from pathlib import Path

In [112]:
def read_notebook_byid(notebook_id, sautsdeligne=True):
    path = f'../raw_data/AI4Code/train_data/{notebook_id}.json'
    df = pd.read_json(path, convert_dates=False).assign(id=notebook_id)
    if sautsdeligne == False:
        df['source'] = df['source'].str.replace('\n', '')
    return df

In [113]:
def get_notebook_w_order(notebook_id, sautsdeligne=True, ordered=False):
    nb = read_notebook_byid(notebook_id=notebook_id, sautsdeligne=sautsdeligne) #lis le nb

    #lis train_orders
    df_orders = pd.read_csv(
    '../raw_data/AI4Code/train_orders.csv',
    index_col='id',
    )

    cell_order = df_orders['cell_order'].loc[notebook_id].split() # liste des cellules dans l'ordre
    cell_ranks = [cell_order.index(e) for e in list(nb.index)] # recupère l'index d'une cellule dans la liste ordonnée
    nb['cell_order'] = cell_ranks

    if ordered :
        nb.sort_values('cell_order', inplace=True)

    return nb

In [114]:
def open_all_notebooks(nb_limit=20, sautsdeligne=True, ordered=False, save_into_csv=False, language='en'):
    all_nb = pd.DataFrame()
    all_ids = pd.read_csv('../raw_data/AI4Code/train_orders.csv')['id'].to_list()[:nb_limit]

    for n in all_ids:
        nb = get_notebook_w_order(notebook_id=n, sautsdeligne=sautsdeligne, ordered=ordered)
        nb = nb.reset_index().rename(columns={'index': 'cell_id'})
        all_nb = pd.concat([all_nb,nb],ignore_index=True)

    ancestors_df = pd.read_csv('../raw_data/AI4Code/train_ancestors.csv').loc[:nb_limit]
    all_nb = pd.merge(all_nb, ancestors_df, how='left', on='id')

    all_languages = pd.read_csv('../raw_data/AI4Code/all_languages.csv')[['id', 'language']].loc[:nb_limit]
    all_nb = pd.merge(all_nb, all_languages, how='left', on='id')

    all_nb['cell_order_training'] = np.where(all_nb['cell_type'] == 'markdown', 0, all_nb['cell_order'])

    if save_into_csv:
        all_nb.to_csv(f'../data/train_data_{language}_{nb_limit}.csv')

    return all_nb[all_nb['language'] == language]

In [115]:
df = open_all_notebooks(nb_limit=20)
df

/tmp/ipykernel_2461/2178489222.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(path, convert_dates=False).assign(id=notebook_id)
/tmp/ipykernel_2461/2178489222.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(path, convert_dates=False).assign(id=notebook_id)
/tmp/ipykernel_2461/2178489222.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetim

,cell_id,cell_type,source,id,cell_order,ancestor_id,parent_id,language,cell_order_training
58,c417225b,code,import numpy as np # linear algebra\nimport pa...,00015c83e2717b,4,aa2da37e,317b65d12af9df,en,4
59,51e3cd89,code,"input_dir = ""../input/""",00015c83e2717b,5,aa2da37e,317b65d12af9df,en,5
60,2600b4eb,code,"train_df = pd.read_csv(""../input/train.csv"")\n...",00015c83e2717b,6,aa2da37e,317b65d12af9df,en,6
61,75b65993,code,class_dict = defaultdict(int)\n\nkind_class_di...,00015c83e2717b,7,aa2da37e,317b65d12af9df,en,7
62,cf195f8b,code,"print(""the number of images with no defects: {...",00015c83e2717b,8,aa2da37e,317b65d12af9df,en,8
...,...,...,...,...,...,...,...,...,...
1056,00c42611,code,from sklearn.ensemble import RandomForestClass...,000af347c31e72,10,e94552a4,b20d828276f2af,en,10
1057,c2b7f0c7,code,X_test = scaler.transform(X_test)\ny_pred = fo...,000af347c31e72,11,e94552a4,b20d828276f2af,en,11
1058,cc942436,markdown,## Decision Tree Classifier,000af347c31e72,7,e94552a4,b20d828276f2af,en,0
1059,a01ba7ea,markdown,## EDA,000af347c31e72,1,e94552a4,b20d828276f2af,en,0


## Preprocessing

In [116]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional, for better language support

[nltk_data] Downloading package wordnet to /home/ilhanb11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ilhanb11/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [117]:
def preprocess_text(text):
    # Remove all the special characters
    text = re.sub(r'\W', ' ', text)
    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Converting to Lowercase
    text = text.strip().lower()

    #return text
    # Lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if len(word) > 3]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [118]:
df.source = df.source.apply(preprocess_text)
df

,cell_id,cell_type,source,id,cell_order,ancestor_id,parent_id,language,cell_order_training
58,c417225b,code,import numpy linear algebra import panda set_o...,00015c83e2717b,4,aa2da37e,317b65d12af9df,en,4
59,51e3cd89,code,input_dir input,00015c83e2717b,5,aa2da37e,317b65d12af9df,en,5
60,2600b4eb,code,train_df read_csv input train sample_df read_c...,00015c83e2717b,6,aa2da37e,317b65d12af9df,en,6
61,75b65993,code,class_dict defaultdict kind_class_dict default...,00015c83e2717b,7,aa2da37e,317b65d12af9df,en,7
62,cf195f8b,code,print number image with defect format no_defec...,00015c83e2717b,8,aa2da37e,317b65d12af9df,en,8
...,...,...,...,...,...,...,...,...,...
1056,00c42611,code,from sklearn ensemble import randomforestclass...,000af347c31e72,10,e94552a4,b20d828276f2af,en,10
1057,c2b7f0c7,code,x_test scaler transform x_test y_pred forest_c...,000af347c31e72,11,e94552a4,b20d828276f2af,en,11
1058,cc942436,markdown,decision tree classifier,000af347c31e72,7,e94552a4,b20d828276f2af,en,0
1059,a01ba7ea,markdown,,000af347c31e72,1,e94552a4,b20d828276f2af,en,0


In [119]:
def preprocess_df(df, min_df=0.01):
    df['cell_type'] = df['cell_type'].map({'code': 0, 'markdown': 1})

    df['cell_count'] = df.groupby(by=['id'])['cell_id'].transform('count')
    df['cell_order_training'] = df['cell_order_training'] / df['cell_count']

    df['source'] = df['source'].apply(lambda x:preprocess_text(x))

    df['markdown_count'] = df.groupby(by=['id'])['cell_type'].transform('sum')
    df = df[df['markdown_count'] > 0]

    df = df.drop(columns=['markdown_count', 'cell_count'])

    df = df.reset_index().drop(columns='index')

    vectorizer = TfidfVectorizer(min_df=min_df)
    source_vect = vectorizer.fit_transform(df['source'])
    source_vect = pd.DataFrame(
        source_vect.toarray(),
        columns = vectorizer.get_feature_names_out())

    df = pd.concat([df, source_vect], axis=1)

    return df

In [120]:
df = preprocess_df(df)
df

,cell_id,cell_type,source,id,cell_order,ancestor_id,parent_id,language,cell_order_training,1000,...,xlabel,xlim,y_pred,y_test,y_train,year,yf_test,ylabel,your,zero
0,c417225b,0,import numpy linear algebra import panda set_o...,00015c83e2717b,4,aa2da37e,317b65d12af9df,en,0.043011,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51e3cd89,0,input_dir input,00015c83e2717b,5,aa2da37e,317b65d12af9df,en,0.053763,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2600b4eb,0,train_df read_csv input train sample_df read_c...,00015c83e2717b,6,aa2da37e,317b65d12af9df,en,0.064516,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,75b65993,0,class_dict defaultdict kind_class_dict default...,00015c83e2717b,7,aa2da37e,317b65d12af9df,en,0.075269,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cf195f8b,0,print number image with defect format no_defec...,00015c83e2717b,8,aa2da37e,317b65d12af9df,en,0.086022,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,00c42611,0,from sklearn ensemble import randomforestclass...,000af347c31e72,10,e94552a4,b20d828276f2af,en,0.833333,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
954,c2b7f0c7,0,x_test scaler transform x_test y_pred forest_c...,000af347c31e72,11,e94552a4,b20d828276f2af,en,0.916667,0.0,...,0.0,0.0,0.408872,0.0,0.0,0.0,0.0,0.0,0.0,0.0
955,cc942436,1,decision tree classifier,000af347c31e72,7,e94552a4,b20d828276f2af,en,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
956,a01ba7ea,1,,000af347c31e72,1,e94552a4,b20d828276f2af,en,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Split

In [121]:
df = df.set_index(['id', 'cell_id'])
df

cell_type  \
id             cell_id               
00015c83e2717b c417225b          0   
               51e3cd89          0   
               2600b4eb          0   
               75b65993          0   
               cf195f8b          0   
...                            ...   
000af347c31e72 00c42611          0   
               c2b7f0c7          0   
               cc942436          1   
               a01ba7ea          1   
               56806367          1   

                                                                    source  \
id             cell_id                                                       
00015c83e2717b c417225b  import numpy linear algebra import panda set_o...   
               51e3cd89                                    input_dir input   
               2600b4eb  train_df read_csv input train sample_df read_c...   
               75b65993  class_dict defaultdict kind_class_dict default...   
               cf195f8b  print number image with defect format no_defec...   
...                                                                    ...   
000af347c31e72 00c42611  from sklearn ensemble import randomforestclass...   
               c2b7f0c7  x_test scaler transform x_test y_pred forest_c...   
               cc942436                           decision tree classifier   
               a01ba7ea                                                      
               56806367                           random forest classifier   

                         cell_order ancestor_id       parent_id language  \
id             cell_id                                                     
00015c83e2717b c417225b           4    aa2da37e  317b65d12af9df       en   
               51e3cd89           5    aa2da37e  317b65d12af9df       en   
               2600b4eb           6    aa2da37e  317b65d12af9df       en   
               75b65993           7    aa2da37e  317b65d12af9df       en   
               cf195f8b           8    aa2da37e  317b65d12af9df       en   
...                             ...         ...             ...      ...   
000af347c31e72 00c42611          10    e94552a4  b20d828276f2af       en   
               c2b7f0c7          11    e94552a4  b20d828276f2af       en   
               cc942436           7    e94552a4  b20d828276f2af       en   
               a01ba7ea           1    e94552a4  b20d828276f2af       en   
               56806367           9    e94552a4  b20d828276f2af       en   

                         cell_order_training  1000  2017  2018  ...  xlabel  \
id             cell_id                                          ...           
00015c83e2717b c417225b             0.043011   0.0   0.0   0.0  ...     0.0   
               51e3cd89             0.053763   0.0   0.0   0.0  ...     0.0   
               2600b4eb             0.064516   0.0   0.0   0.0  ...     0.0   
               75b65993             0.075269   0.0   0.0   0.0  ...     0.0   
               cf195f8b             0.086022   0.0   0.0   0.0  ...     0.0   
...                                      ...   ...   ...   ...  ...     ...   
000af347c31e72 00c42611             0.833333   0.0   0.0   0.0  ...     0.0   
               c2b7f0c7             0.916667   0.0   0.0   0.0  ...     0.0   
               cc942436             0.000000   0.0   0.0   0.0  ...     0.0   
               a01ba7ea             0.000000   0.0   0.0   0.0  ...     0.0   
               56806367             0.000000   0.0   0.0   0.0  ...     0.0   

                         xlim    y_pred  y_test  y_train  year  yf_test  \
id             cell_id                                                    
00015c83e2717b c417225b   0.0  0.000000     0.0      0.0   0.0      0.0   
               51e3cd89   0.0  0.000000     0.0      0.0   0.0      0.0   
               2600b4eb   0.0  0.000000     0.0      0.0   0.0      0.0   
               75b65993   0.0  0.000000     0.0      0.0   0.0      0.0   
               cf195f8b   0.0  0.000000     0.0 

In [122]:
print(df.index.names)

['id', 'cell_id']


In [123]:
ancestors_df = pd.read_csv('../raw_data/AI4Code/train_ancestors.csv').loc[:20]

In [125]:
from sklearn.model_selection import GroupShuffleSplit

In [126]:
def train_val_split(df, test_size=0.1):
    df = df.reset_index(drop=True).set_index('id')
    splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=0)
    # Split, keeping notebooks with a common origin (ancestor_id) together
    ids = df.index.unique('id')
    ancestors = df.loc[ids, 'ancestor_id'].drop_duplicates()
    ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
    ids_train, ids_valid = ids[ids_train], ids[ids_valid]
    df_train = df.loc[ids_train, :]
    df_valid = df.loc[ids_valid, :]
    return df_train, df_valid

In [129]:
df_train, df_valid = train_val_split(df)

KeyError: "None of ['id'] are in the columns"